In [2]:
import requests
from requests import Response
import pandas as pd
aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
response:Response=requests.get(aqi_url)
if response.status_code==200:
    print('下載成功')
else:
    print('下載失敗')

#得到python的資料結構(list或dict)
data_base:dict=response.json()

#json的字串格式
data_json=response.text

from pydantic import BaseModel, Field, field_validator
class Site(BaseModel):
    站點:str=Field(alias='sitename')
    縣市:str=Field(alias='county')
    aqi:int=Field(alias='aqi')
    pm25:float=Field(alias='pm2.5_avg')
    時間:str=Field(alias='datacreationdate')
    #解決model_validate_json空字串的問題
    @field_validator('pm25', mode='before')
    @classmethod
    def sapce_zero(cls, value):
        if value=='':
            value=0.0
        return value

class AQI(BaseModel):
    records:list[Site] #records名字要和資料的一樣(json viewer)


aqi=AQI.model_validate_json(data_json)
#source_data=aqi.model_dump()
source_data=aqi.model_dump()['records'] #(aqi.model_dump()的key是records)

df1=pd.DataFrame(source_data)

下載成功


In [4]:
#選取多欄位
df2=df1[['縣市', 'pm25', 'aqi']].copy()
df2

,縣市,pm25,aqi
0,屏東縣,0.0,23
1,臺中市,5.0,22
2,新北市,6.3,22
3,屏東縣,9.2,30
4,臺南市,5.0,23
...,...,...,...
995,苗栗縣,7.3,24
996,新竹市,4.0,16
997,新竹縣,5.5,18
998,新竹縣,7.5,24


In [11]:
#iloc
df2.iloc[0]
df2.iloc[[0, 99, 196]]
df2.iloc[0:10]
df3=df2.iloc[-10:].copy()

In [12]:
df3['aqi']

990    16
991    19
992    21
993    18
994    19
995    24
996    16
997    18
998    24
999    18
Name: aqi, dtype: int64

In [14]:
df3.aqi>=20
#得到一個series

990    False
991    False
992     True
993    False
994    False
995     True
996    False
997    False
998     True
999    False
Name: aqi, dtype: bool

In [16]:
#搜尋資料
df3[df3.aqi>=20]

,縣市,pm25,aqi
992,臺中市,4.7,21
995,苗栗縣,7.3,24
998,新竹縣,7.5,24
